In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.SFI_FIM import SFIModel_pipeline,SFIModel_gating,SFIModel_unified

In [2]:
hparams = {
    'scale':'demo',
    'name':'sfi-fim',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'filter_num':150,
    'epochs':10,
    'metrics':'group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'integration':'harmony',
    'k':10,
    'save_step':None,
    'save_each_epoch':False,
    'train_embedding':False
}

device = torch.device(hparams['device'])

In [3]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [4]:
sfiModel_gating = SFIModel_gating(hparams, vocab=vocab).to(device)

In [6]:
train(sfiModel_gating, hparams, loader_train, loader_test, loader_validate)

training...
epoch 1 , step 190 , loss: 1.5481: : 200it [00:11, 17.77it/s]
epoch 2 , step 190 , loss: 1.4636: : 200it [00:10, 18.50it/s]
epoch 3 , step 190 , loss: 1.4078: : 200it [00:10, 18.95it/s]
epoch 4 , step 190 , loss: 1.3435: : 200it [00:10, 19.10it/s]
epoch 5 , step 190 , loss: 1.2842: : 200it [00:10, 19.04it/s]
epoch 6 , step 190 , loss: 1.2278: : 200it [00:10, 19.14it/s]
epoch 7 , step 190 , loss: 1.2054: : 200it [00:10, 19.11it/s]
epoch 8 , step 190 , loss: 1.1468: : 200it [00:10, 19.11it/s]
epoch 9 , step 190 , loss: 1.1057: : 200it [00:10, 19.00it/s]
epoch 10 , step 190 , loss: 1.0459: : 200it [00:10, 19.18it/s]
0it [00:00, ?it/s]testing...
1811it [00:41, 43.70it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.5344, 'ndcg@5': 0.253, 'ndcg@10': 0.3183, 'mean_mrr': 0.2377}
validating...
7531it [02:49, 44.42it/s]
evaluation results:{'group_auc': 0.8349, 'ndcg@5': 0.5501, 'ndcg@10': 0.5959, 'mean_mrr': 0.4988}


In [4]:
sfiModel_pipeline = SFIModel_pipeline(hparams, vocab=vocab).to(device)

In [5]:
train(sfiModel_pipeline, hparams, loader_train, loader_test, loader_validate)

training...
epoch 1 , step 190 , loss: 1.6079: : 200it [00:05, 38.62it/s]
epoch 2 , step 190 , loss: 1.5874: : 200it [00:04, 41.63it/s]
epoch 3 , step 190 , loss: 1.5785: : 200it [00:04, 44.47it/s]
epoch 4 , step 190 , loss: 1.5661: : 200it [00:04, 42.99it/s]
epoch 5 , step 190 , loss: 1.5526: : 200it [00:04, 43.46it/s]
epoch 6 , step 190 , loss: 1.5363: : 200it [00:04, 41.74it/s]
epoch 7 , step 190 , loss: 1.5381: : 200it [00:04, 43.62it/s]
epoch 8 , step 190 , loss: 1.5146: : 200it [00:04, 42.60it/s]
epoch 9 , step 190 , loss: 1.4995: : 200it [00:04, 43.96it/s]
epoch 10 , step 190 , loss: 1.4976: : 200it [00:04, 42.31it/s]
0it [00:00, ?it/s]testing...
1811it [00:20, 88.40it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.5698, 'ndcg@5': 0.2822, 'ndcg@10': 0.3379, 'mean_mrr': 0.2565}
validating...
7531it [01:23, 90.29it/s]
evaluation results:{'group_auc': 0.662, 'ndcg@5': 0.3738, 'ndcg@10': 0.4294, 'mean_mrr': 0.3471}


In [4]:
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(device)

In [5]:
train(sfiModel_unified, hparams, loader_train, loader_test, loader_validate)

training...
epoch 1 , step 190 , loss: 1.6171: : 200it [00:10, 18.99it/s]
epoch 2 , step 190 , loss: 1.6150: : 200it [00:09, 20.15it/s]
epoch 3 , step 190 , loss: 1.6102: : 200it [00:09, 20.26it/s]
epoch 4 , step 190 , loss: 1.6069: : 200it [00:09, 20.23it/s]
epoch 5 , step 190 , loss: 1.5990: : 200it [00:09, 20.20it/s]
epoch 6 , step 190 , loss: 1.5964: : 200it [00:09, 20.45it/s]
epoch 7 , step 190 , loss: 1.5810: : 200it [00:09, 20.02it/s]
epoch 8 , step 190 , loss: 1.5674: : 200it [00:09, 20.35it/s]
epoch 9 , step 190 , loss: 1.5469: : 200it [00:09, 20.68it/s]
epoch 10 , step 190 , loss: 1.5197: : 200it [00:09, 20.09it/s]
0it [00:00, ?it/s]testing...
1811it [00:37, 48.93it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.501, 'ndcg@5': 0.2232, 'ndcg@10': 0.2849, 'mean_mrr': 0.2132}
validating...
7531it [02:32, 49.47it/s]
evaluation results:{'group_auc': 0.6477, 'ndcg@5': 0.3603, 'ndcg@10': 0.4122, 'mean_mrr': 0.3402}


In [6]:
hparams['integration'] = 'gate'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(device)

In [7]:
train(sfModel_unified, hparams, loader_train, loader_test, loader_validate)

training...
epoch 1 , step 190 , loss: 1.5371: : 200it [00:09, 20.89it/s]
epoch 2 , step 190 , loss: 1.4665: : 200it [00:09, 20.15it/s]
epoch 3 , step 190 , loss: 1.4073: : 200it [00:08, 23.69it/s]
epoch 4 , step 190 , loss: 1.3886: : 200it [00:10, 19.96it/s]
epoch 5 , step 190 , loss: 1.3242: : 200it [00:10, 19.83it/s]
epoch 6 , step 190 , loss: 1.3090: : 200it [00:09, 20.22it/s]
epoch 7 , step 190 , loss: 1.2726: : 200it [00:09, 20.09it/s]
epoch 8 , step 190 , loss: 1.2539: : 200it [00:10, 19.84it/s]
epoch 9 , step 190 , loss: 1.2028: : 200it [00:09, 20.04it/s]
epoch 10 , step 190 , loss: 1.1305: : 200it [00:09, 20.02it/s]
5it [00:00, 47.47it/s]testing...
1811it [00:36, 50.03it/s]
5it [00:00, 48.43it/s]evaluation results:{'group_auc': 0.558, 'ndcg@5': 0.2602, 'ndcg@10': 0.3238, 'mean_mrr': 0.2393}
validating...
7531it [02:31, 49.73it/s]
evaluation results:{'group_auc': 0.8044, 'ndcg@5': 0.5032, 'ndcg@10': 0.5479, 'mean_mrr': 0.451}
